In [1]:
import pandas as pd
from cryptography.fernet import Fernet
import logging
from datetime import datetime
import os

# Configuração do logger - Caminho para salvar o log no DBFS
log_path = "/dbfs/logs/silver.log" 
os.makedirs(os.path.dirname(log_path), exist_ok=True)

logging.basicConfig(
    filename=log_path,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

def log_info(message):
    logging.info(message)

def log_error(message):
    logging.error(message)

  # Transformações para a camada Silver
def ajustar_dados_silver(df):
    log_info("Ajustando dados para a camada Silver.")
    df['Open'] = df['Open'].round(2)
    df['Close'] = df['Close'].round(2)
    df['High'] = df['High'].round(2)
    df['Low'] = df['Low'].round(2)
    df['Volume'] = df['Volume'].astype(int)
    df['Ticker'] = df['Ticker'].apply(lambda x: cipher.encrypt(x.encode()).decode())
    df.drop(columns=['Capital Gains'], inplace=True)
    return df

    # log_info("Dados ajustados para a camada Silver.")

    # # Salvar na camada Silver
    # silver_path = "/dbfs/silver/dados_silver.parquet"
    # # dados_silver.to_parquet(silver_path, index=False)
    # dados_silver.to_csv("dados_silver.csv", index=False)
    # log_info(f"Dados salvos na camada Silver: {silver_path}")


In [3]:
dados_bronze = pd.read_csv('dados_acoes_ativas.csv')

In [5]:
dados_silver = ajustar_dados_silver(dados_bronze)

NameError: name 'cipher' is not defined

In [7]:
import pandas as pd
from cryptography.fernet import Fernet
import logging
import os

# Configuração do logger
log_path = "/dbfs/logs/silver.log"
os.makedirs(os.path.dirname(log_path), exist_ok=True)

logging.basicConfig(
    filename=log_path,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

def log_info(message):
    logging.info(message)

def log_error(message):
    logging.error(message)

try:
    log_info("Início da execução da camada Silver.")

    # Verificar se a chave de criptografia existe
    key_path = "/dbfs/chave/chave.key"
    if not os.path.exists(key_path):
        log_info("Arquivo de chave não encontrado. Gerando nova chave de criptografia.")
        key = Fernet.generate_key()
        with open(key_path, "wb") as key_file:
            key_file.write(key)
        log_info(f"Nova chave de criptografia gerada e salva em: {key_path}")
    else:
        log_info("Carregando chave de criptografia existente.")
        with open(key_path, "rb") as key_file:
            key = key_file.read()

    # Criar o objeto de criptografia 
    # este é em apontado para o DBFS    
    cipher = Fernet(key)

    # Leitura da camada Bronze
    log_info("Carregando dados da camada Bronze.")
    dados_bronze = pd.read_parquet("/dbfs/bronze/dados_bronze.parquet")
    log_info(f"Dados carregados: {len(dados_bronze)} registros encontrados.")

    # Transformações para a camada Silver
    def ajustar_dados_silver(df):
        log_info("Ajustando dados para a camada Silver.")
        
        # Ajuste de valores numéricos
        df['Open'] = df['Open'].round(2)
        df['Close'] = df['Close'].round(2)
        df['High'] = df['High'].round(2)
        df['Low'] = df['Low'].round(2)
        df['Volume'] = df['Volume'].astype(int)
        #Dropa coluna vazia
        df.drop(columns=['Capital Gains'], inplace=True)
        # Criptografar o ticker
        df['Ticker'] = df['Ticker'].apply(lambda x: cipher.encrypt(x.encode()).decode())
        return df

    dados_silver = ajustar_dados_silver(dados_bronze)
    log_info("Dados ajustados para a camada Silver.")

    # Salvar na camada Silver
    silver_path = "/dbfs/silver/dados_silver.parquet"
    dados_silver.to_parquet(silver_path, index=False)
    log_info(f"Dados salvos na camada Silver: {silver_path}")

    log_info("Camada Silver executada com sucesso.")

except Exception as e:
    log_error(f"Erro na execução da camada Silver: {e}")
    raise e


FileNotFoundError: [Errno 2] No such file or directory: '/dbfs/chave/chave.key'

In [9]:
#local
def get_or_create_key(key_path="chave.key"):
    if not os.path.exists(key_path):
        # Gerar uma nova chave de criptografia
        key = Fernet.generate_key()
        with open(key_path, "wb") as key_file:
            key_file.write(key)
    else:
        # Carregar a chave existente
        with open(key_path, "rb") as key_file:
            key = key_file.read()
    return Fernet(key)

# Inicializando o cipher
cipher = get_or_create_key()

In [11]:
dados_silver

NameError: name 'dados_silver' is not defined

In [184]:
# Carregar os dados criptografados TESTAR LOCAL
# encrypted_df = pd.read_csv("chave.key")
encrypted_df = dados_silver

def load_cipher(key_path="chave.key"):
    if not os.path.exists(key_path):
        raise FileNotFoundError(f"Chave de criptografia não encontrada: {key_path}")
    with open(key_path, "rb") as key_file:
        key = key_file.read()
    return Fernet(key)

# Inicializar o cipher para a camada Gold
key_path = "chave.key"
cipher = load_cipher(key_path)



In [186]:
# Descriptografar a coluna 'Ticker'
def descriptograr_coluna(df, encrypted_column, new_column_name):
    df[new_column_name] = df[encrypted_column].apply(lambda x: cipher.decrypt(x.encode()).decode())
    return df

# Reverter a criptografia
dados_gold = descriptograr_coluna(dados_silver, "Ticker", "Ticker")
print("Dados descriptografados:")
print(decrypted_df)

Dados descriptografados:
           Open     High      Low    Close   Volume  Dividends  Stock Splits  \
0         19.00    19.00    18.44    18.44   716674        0.0           0.0   
1         18.32    18.36    17.62    17.78  1660144        0.0           0.0   
2         17.78    18.12    17.43    18.03  1090862        0.0           0.0   
3         17.90    18.03    17.35    17.49  1145904        0.0           0.0   
4         17.10    17.84    17.07    17.78   946834        0.0           0.0   
...         ...      ...      ...      ...      ...        ...           ...   
283763  1059.99  1059.99  1059.99  1059.99        0        0.0           0.0   
283764  1059.99  1059.99  1059.99  1059.99        0        0.0           0.0   
283765  1059.99  1059.99  1059.99  1059.99        0        0.0           0.0   
283766  1059.99  1059.99  1059.99  1059.99        0        0.0           0.0   
283767  1059.99  1059.99  1059.99  1059.99        0        0.0           0.0   

           Tic